In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler

In [2]:
spark = SparkSession.builder.appName('project2').getOrCreate()

In [3]:
data = spark.read.csv('Dataset/customer_churn.csv', inferSchema=True, header=True)

In [4]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [6]:
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [7]:
assembler = VectorAssembler(inputCols=['Age','Total_Purchase','Num_Sites'],outputCol='features')

In [8]:
output = assembler.transform(data)

In [9]:
final_data= output.select('features','churn')

In [10]:
train_data , test_data = final_data.randomSplit([0.7,0.3])

In [11]:
log_reg_churn = LogisticRegression(labelCol='churn')

In [12]:
fitted_churn_model = log_reg_churn.fit(train_data)

In [13]:
training_sum = fitted_churn_model.summary

In [14]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                643|                643|
|   mean|0.16329704510108864| 0.1290824261275272|
| stddev| 0.3699242371375314|0.33555217577264485|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [15]:
pred_and_labels = fitted_churn_model.evaluate(test_data)

In [16]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
| [22.0,11254.38,8.0]|    0|[3.80988548612187...|[0.97832930599869...|       0.0|
| [28.0,9090.43,10.0]|    0|[1.31421706778981...|[0.78821796831099...|       0.0|
|  [29.0,9378.24,8.0]|    0|[3.60230000424554...|[0.97346248780684...|       0.0|
|  [29.0,9617.59,8.0]|    0|[3.60814658093113...|[0.97361310665406...|       0.0|
| [29.0,10203.18,8.0]|    0|[3.62245072482630...|[0.97397811006713...|       0.0|
| [29.0,12711.15,7.0]|    0|[4.83579304090848...|[0.99212216430386...|       0.0|
| [29.0,13240.01,6.0]|    0|[6.00079185005855...|[0.99752932917963...|       0.0|
| [29.0,13255.05,8.0]|    0|[3.69699842480302...|[0.97580220573312...|       0.0|
| [30.0,10960.52,9.0]|    0|[2.46576161147544...|[0.92170645336906...|       0.0|
|[30.0,13473.35,

In [17]:
churm_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [18]:
auc = churm_eval.evaluate(pred_and_labels.predictions)

In [19]:
auc

0.7272536687631027

## Predict on New Data

In [20]:
final_lr_model = log_reg_churn.fit(final_data)

In [21]:
new_customers = spark.read.csv('Dataset/new_customers.csv', inferSchema=True, header=True)

In [22]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [23]:
test_new_customers = assembler.transform(new_customers)

In [24]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [25]:
final_results = final_lr_model.transform(test_new_customers)

In [26]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [27]:
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                null|            null|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|                null|            null|
|    min|Andrew Mccall|              22.0|            100.0|          